In [18]:
from random import randint
from time import sleep
import datetime
import yfinance as yf
import openpyxl
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
%matplotlib inline
from mystockrec import xlsx_df

In [23]:
df = xlsx_df('df_k_687_2021.10.06 01_34_53.xlsx')
display(df.loc[0:3,'Unnamed: 0'])
df.head()

0    1
1    3
2    4
3    5
Name: Unnamed: 0, dtype: int64

,Unnamed: 0,shortName,symbol,financialCurrency,returnOnAssets,revenueGrowth,grossMargins,ebitdaMargins,trailingPE,beta,cluster
0,1,DUERR AG O.N.,DUE.DE,EUR,0.01104,0.091,0.20667,0.04322,125.410960,1.750285,6
1,3,COMPUGROUP MED. NA O.N.,COP.DE,EUR,0.04650,0.316,0.37123,0.17535,52.999240,0.836374,0
2,4,NFON AG INH O.N.,NFN.DE,EUR,-0.01891,0.160,0.38878,0.01379,52.999240,1.646321,6
3,5,MEDARTIS N,MED.SW,CHF,0.02513,0.291,0.83734,0.14498,105.835800,0.771998,2
4,6,SAP SE,SAP.DE,EUR,0.06056,-0.011,0.71777,0.26134,23.425348,0.927594,2


In [28]:
#clean data set: in "df_spotify" and "df" we lower cases for song title and clean it from special characters.
df['shortName'] = df['shortName'].apply(lambda x: x.upper().translate({ord(c): "" for c in "!?/[]|{}.:()'\""}))

In [53]:
df.columns

Index(['Unnamed: 0', 'shortName', 'symbol', 'financialCurrency',
       'returnOnAssets', 'revenueGrowth', 'grossMargins', 'ebitdaMargins',
       'trailingPE', 'beta', 'cluster'],
      dtype='object')

In [29]:
def get_input(): 
    shortName_input = input("What company do you want to check? ")
    shortName_input = shortName_input.upper()
    comp_match = df[df['shortName'].str.contains(shortName_input)]
    if len(comp_match) > 0:
        print("The following company was found in the database, let's take a closer look.")
        display(comp_match)
        comp_match['cluster']
    return shortName_input

In [52]:
get_input()

What company do you want to check? pepsi
The following company was found in the database, let's take a closer look.


,Unnamed: 0,shortName,symbol,financialCurrency,returnOnAssets,revenueGrowth,grossMargins,ebitdaMargins,trailingPE,beta,cluster
568,609,"PEPSICO, INC",PEP,USD,0.08035,0.205,0.54295,0.18371,27.79341,0.614846,5


'PEPSI'

In [85]:
shortName_input = input("What company do you want to check? ")
shortName_input = shortName_input.upper()
comp_match = df[df['shortName'].str.contains(shortName_input)]
if len(comp_match) > 0:
    print("The following company was found in the database, let's take a closer look.")
    display(comp_match)
    print(comp_match.loc[1:,'shortName'],"is in cluster",comp_match.loc[1:,'cluster'],"in industry",
          comp_match.loc[1:,'symbol'],"and was ranked on overall position",comp_match.loc[1:,'beta'],".")
    print("Top ranked companies in cluster",comp_match.loc[1:,'cluster'],":")
    comp_match['cluster']
    display(df[df['cluster'].isin([int(comp_match.loc[1:,'cluster'])])].nlargest(10,['beta']))

What company do you want to check? apple
The following company was found in the database, let's take a closer look.


,Unnamed: 0,shortName,symbol,financialCurrency,returnOnAssets,revenueGrowth,grossMargins,ebitdaMargins,trailingPE,beta,cluster
315,342,APPLE INC,AAPL,USD,0.19302,0.364,0.41005,0.31955,27.222006,1.222222,5


315    APPLE INC
Name: shortName, dtype: object is in cluster 315    5
Name: cluster, dtype: int64 in industry 315    AAPL
Name: symbol, dtype: object and was ranked on overall position 315    1.222222
Name: beta, dtype: float64 .
Top ranked companies in cluster 315    5
Name: cluster, dtype: int64 :


,Unnamed: 0,shortName,symbol,financialCurrency,returnOnAssets,revenueGrowth,grossMargins,ebitdaMargins,trailingPE,beta,cluster
39,42,DEUTSCHE EUROSHOP NA ON,DEQ.DE,EUR,0.03706,4.491,0.90585,0.88364,23.364828,2.031356,5
615,658,"SIMON PROPERTY GROUP, INC",SPG,USD,0.03761,0.181,0.80963,0.71810,37.166573,1.552947,5
226,252,"TREX COMPANY, INC",TREX,USD,0.21336,0.412,0.39082,0.28840,64.515920,1.513100,5
316,343,"APPLIED MATERIALS, INC",AMAT,USD,0.17930,0.410,0.46749,0.32060,21.747220,1.431216,5
29,31,"AROUNDTOWN EO-,01",AT1.DE,EUR,0.01887,-0.162,0.75992,0.72693,16.366577,1.413970,5
158,176,ADAIRS FPO,ADH.AX,AUD,0.16625,0.231,0.60687,0.23238,10.052356,1.376852,5
556,596,NVIDIA CORPORATION,NVDA,USD,0.14360,0.683,0.63758,0.38759,23.276596,1.359716,5
229,255,UBIQUITI INC,UI,USD,0.57002,0.515,0.48063,0.39761,60.358208,1.355349,5
590,631,"QORVO, INC",QRVO,USD,0.10108,0.410,0.48498,0.36154,25.904827,1.354803,5
181,207,LULULEMON ATHLETICA INC,LULU,USD,0.16255,0.881,0.56867,0.23548,74.479490,1.340760,5


In [83]:
# Filter titanic[titanic["Pclass"].isin([2, 3])] - 
# https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html
cluster_val = comp_match.loc[1:,'cluster']
comp_name = str(comp_match.loc[1:,'shortName'])
print(comp_match.loc[1:,'shortName'])
type(comp_name)
df[df['cluster'].isin([int(comp_match.loc[1:,'cluster'])])].nlargest(10,['beta'])

315    APPLE INC
Name: shortName, dtype: object


,Unnamed: 0,shortName,symbol,financialCurrency,returnOnAssets,revenueGrowth,grossMargins,ebitdaMargins,trailingPE,beta,cluster
39,42,DEUTSCHE EUROSHOP NA ON,DEQ.DE,EUR,0.03706,4.491,0.90585,0.88364,23.364828,2.031356,5
615,658,"SIMON PROPERTY GROUP, INC",SPG,USD,0.03761,0.181,0.80963,0.71810,37.166573,1.552947,5
226,252,"TREX COMPANY, INC",TREX,USD,0.21336,0.412,0.39082,0.28840,64.515920,1.513100,5
316,343,"APPLIED MATERIALS, INC",AMAT,USD,0.17930,0.410,0.46749,0.32060,21.747220,1.431216,5
29,31,"AROUNDTOWN EO-,01",AT1.DE,EUR,0.01887,-0.162,0.75992,0.72693,16.366577,1.413970,5
158,176,ADAIRS FPO,ADH.AX,AUD,0.16625,0.231,0.60687,0.23238,10.052356,1.376852,5
556,596,NVIDIA CORPORATION,NVDA,USD,0.14360,0.683,0.63758,0.38759,23.276596,1.359716,5
229,255,UBIQUITI INC,UI,USD,0.57002,0.515,0.48063,0.39761,60.358208,1.355349,5
590,631,"QORVO, INC",QRVO,USD,0.10108,0.410,0.48498,0.36154,25.904827,1.354803,5
181,207,LULULEMON ATHLETICA INC,LULU,USD,0.16255,0.881,0.56867,0.23548,74.479490,1.340760,5


In [71]:
df.shape

(687, 11)

In [ ]:
#Check if song is in large DB/ if input makes any sense
def comp_check():
    comp_check = df_spotify[df_spotify['track_name'].str.contains(song_input)]
    if len(song_check0) > 0:
        return song_check0 
    elif len(comp_match) > 1:
    else:
        get_input()
song_check0 = song_check_overall()

In [ ]:
# df['symbol']
# user input = 'AAPL'
# df.loc[df['symbol']==user input]
# if len(df.loc[df['symbol']==user input]) ==1
#    print('The following company was found:')
#    elif len(df.loc[df['symbol']==user input]) > 1
# df = df.loc

In [ ]:
# Selection idea
def user_selection():
    for index, item in enumerate(options):
                print(f"Option {index + 1} - {item}")
            while True:
                try: # Is used to avoid errors
                    choice = int(input("Selection: "))
                    if choice >= 1 and choice <= len(options):
                        return options[choice-1]
                except:
                    continue